# BEE 4750 Homework 5: Mixed Integer and Stochastic Programming

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 12/05/24, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will use mixed integer programming to solve a
    waste load allocation problem.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/Desktop/Julia Scripts/hw5-team-auk`
   Installed JpegTurbo_jll ──────── v3.0.4+0
   Installed GR_jll ─────────────── v0.73.8+0
   Installed LERC_jll ───────────── v4.0.0+0
   Installed LoggingExtras ──────── v1.1.0
   Installed OffsetArrays ───────── v1.14.1
   Installed MutableArithmetics ─── v1.5.2
   Installed NetworkLayout ──────── v0.4.7
   Installed Cairo_jll ──────────── v1.18.2+1
   Installed PlotUtils ──────────── v1.4.3
   Installed StaticArrays ───────── v1.9.8
   Installed HTTP ───────────────── v1.10.10
   Installed Libgpg_error_jll ───── v1.50.0+0
   Installed HiGHS_jll ──────────── v1.8.1+0
   Installed Bzip2_jll ──────────── v1.0.8+2
   Installed FFMPEG ─────────────── v0.4.2
   Installed SentinelArrays ─────── v1.4.7
   Installed DataFrames ─────────── v1.7.0
   Installed CodecBzip2 ─────────── v0.8.4
   Installed XZ_jll ─────────────── v5.6.3+0
   Installed PlotThemes ─────────── v3.3.0
   Installed LZO_jll ────────────── v2.10.2+1
   Instal

In [3]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Problems (Total: 30 Points)

### Problem 1 (30 points)

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided below.

-   **LF**: Capacity 200 Mg, fixed cost \$2000/day, tipping cost
    \$50/Mg;
-   **MRF**: Capacity 350 Mg, fixed cost \$1500/day, tipping cost
    \$7/Mg, recycling cost \$40/Mg recycled;
-   **WTE**: Capacity 210 Mg, fixed cost \$2500/day, tipping cost
    \$60/Mg;

The MRF recycling rate is 40%, and the ash fraction of non-recycled
waste is 16% and of recycled waste is 14%. Transportation costs are
\$1.5/Mg-km, and the relative distances between the cities and
facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

**In this problem**:

-   Formulate the waste load allocation problem and implement it in
    `JuMP`.
-   Draw a diagram showing the flows of waste between the cities and the
    facilities. Which facilities (if any) will not be used? Does this
    solution make sense?

In [138]:
I = 1:3
J = 1:3
A = 1:3

wasteDist_model = Model(HiGHS.Optimizer)
@variable(wasteDist_model, X[i in I, j in J] >= 0)
@variable(wasteDist_model, Y[a in A] >= 0)
@variable(wasteDist_model, ON[j in J], Bin)
@objective(wasteDist_model, Min, 1500*ON[1]+2500*ON[2]+2000*ON[3]+68*X[1,1]+82.5*X[1,2]+57.5*X[1,3]
    +60.5*X[2,1]+75*X[2,2]+72.5*X[2,3]+90.5*X[3,1]+90*X[3,2]+69.5*X[3,3]+82.5*Y[1]
    +77*Y[2]+98*Y[3])
@constraint(wasteDist_model, outFlow_1, X[1,1]+X[1,2]+X[1,3] == 100)
@constraint(wasteDist_model, outFlow_2, X[2,1]+X[2,2]+X[2,3] == 90)
@constraint(wasteDist_model, outFlow_3, X[3,1]+X[3,2]+X[3,3] == 120)
@constraint(wasteDist_model, sizeMRF, X[1,1]+X[2,1]+X[3,1] <= 350)
@constraint(wasteDist_model, sizeWTE, X[1,2]+X[2,2]+X[3,2]+Y[1] <= 210)
@constraint(wasteDist_model, sizeLF, X[1,3]+X[2,3]+X[3,3]+Y[3]+Y[2] == 200)

@constraint(wasteDist_model, outFlow_MRF, Y[1]+Y[3]-0.6(X[1,1]+X[2,1]+X[3,1]) == 0)
@constraint(wasteDist_model, outflow_WTE, Y[2]-0.16(X[1,2]+X[2,2]+X[3,2])-0.14(Y[1]) == 0)

@constraint(wasteDist_model, commit_MRF, 50000*ON[1]-X[1,1]-X[2,1]-X[3,1] >= 0)
@constraint(wasteDist_model, commit_WTE, 50000*ON[2]-X[1,2]-X[2,2]-X[3,2]-Y[1] >= 0)
@constraint(wasteDist_model, commit_LF, ON[3]==1)

optimize!(wasteDist_model)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-01, 5e+04]
  Cost   [6e+01, 2e+03]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 4e+02]
Presolving model
9 rows, 14 cols, 37 nonzeros  0s
8 rows, 13 cols, 35 nonzeros  0s

Solving MIP model with:
   8 rows
   13 cols (2 binary, 0 integer, 0 implied int., 11 continuous)
   35 nonzeros
MIP-Timing:     0.00038 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

       

In [139]:
@show objective_value(wasteDist_model)
@expression(wasteDist_model, total_City1, X[1,1]+X[1,2]+X[1,3])
@show value.(total_City1)

objective_value(wasteDist_model) = 27792.857142857145
value.(total_City1) = 100.0


100.0

In [140]:
@expression(wasteDist_model, total_City2, X[2,1]+X[2,2]+X[2,3])
@show value.(total_City2)

@expression(wasteDist_model, total_City3, X[3,1]+X[3,2]+X[3,3])
@show value.(total_City3)

@expression(wasteDist_model, total_MRF_input, X[1,1]+X[2,1]+X[3,1])
@show value.(total_MRF_input)

@expression(wasteDist_model, total_WTE_input, X[1,2]+X[2,2]+X[3,2]+Y[1])
@show value.(total_WTE_input)

@expression(wasteDist_model, total_LF_input, X[1,3]+X[2,3]+X[3,3]+Y[3]+Y[2])
@show value.(total_LF_input)


value.(total_City2) = 90.0
value.(total_City3) = 120.0
value.(total_MRF_input) = 0.0
value.(total_WTE_input) = 130.95238095238096
value.(total_LF_input) = 200.0


200.0

In [146]:
for i = 1:3
    for j = 1:3
        @show value.(X[i,j])
    end
end

for j = 1:3
    @show value.(Y[j])
    @show value.(ON[j])
end


value.(X[i, j]) = 0.0
value.(X[i, j]) = 0.0
value.(X[i, j]) = 100.0
value.(X[i, j]) = -0.0
value.(X[i, j]) = 90.0
value.(X[i, j]) = -0.0
value.(X[i, j]) = 0.0
value.(X[i, j]) = 40.95238095238096
value.(X[i, j]) = 79.04761904761904
value.(Y[j]) = 0.0
value.(ON[j]) = -0.0
value.(Y[j]) = 20.952380952380963
value.(ON[j]) = 1.0
value.(Y[j]) = 0.0
value.(ON[j]) = 1.0


## References

List any external references consulted, including classmates.

For this model, I used the JuMP manual at
https://jump.dev/JuMP.jl/stable/manual/constraints/
This helped me understand how to make the indicator variables into Boolean 1/0 values only.